# <center> Générateurs de nombres aléatoires <br> TP3 - Générateur à signal d'arrêt et générateur de Geffe  </center>
<center> 2023/2024 - L. Naert/A.Ridard </center>

In [29]:
import numpy.random as npr
import numpy as np
import itertools as it

In [30]:
# Fonctions utiles du TP2 :
def etatSuivant(etat,iCoeff):
    bit_suiv = 0
    sortie = 0
    try:
        assert len(iCoeff) == 0 or max(iCoeff) < len(etat)
        for i in iCoeff:
            bit_suiv = bit_suiv ^ etat[i] #XOR sur les bits concernés
        sortie = etat[0]
        reg = etat[1:]+[bit_suiv] # on décale le registre
        return reg, sortie
    except:
        print("erreur etatSuivant : la taille du registre etat ne correspond pas aux coefficients fournis")
        
def convertToList(suiteCaractere):
    return [int(i) for i in list(suiteCaractere)]


def suite_LFSR(graine,iCoeff,n):
    res = ""
    etatReg = graine
    for i in range(n) :
        etatReg, sortie = etatSuivant(etatReg,iCoeff)
        res = res + str(sortie)
    return convertToList(res)


print(suite_LFSR([1,0,0,1,0,1],[0,3,5], 10))

[1, 0, 0, 1, 0, 1, 1, 1, 0, 0]


## Exercice 1 : le générateur à signal d'arrêt 

Le générateur à signal d'arrêt est un exemple de registre à décalage irrégulier (1984). Il utilise la sortie d'un premier LFSR $R_1$ pour contrôler l'horloge d'un second LFSR $R_2$. Plus précisément, $R_2$ ne change d'état à l'instant $t$ que si la sortie de $R_1$ est égale à 1 à l'instant $t-1$. Si la sortie de $R_1$ est égale à 0 à l'instant $t-1$, alors $R_2$ n'est pas décalé et le bit de sortie à l'instant $t$ est identique au bit de sortie à l'instant $t-1$.

<img src="GSA_TP3.png" width="600">

> **Question 0 (à la main) :**

> En supposant que $R_1$ et $R_2$ produisent des séquences uniformément distribuées, calculer la probabilité que deux bits consécutifs produits par le générateur à signal d'arrêt soient égaux.

$P(R_2(t) = R_2(t-1))

**Réponse :**


> **Question 1 :**

1. Définir une fonction `generateurSignalArret(g1, g2, iCoeff1, iCoeff2, n)` permettant de simuler un tel générateur. `g1` et `g2` sont les graines des deux registres. `iCoeff1` et `iCoeff2` sont les coefficients de la fonction de rétroaction (cf. TP2).
2. Définir une fonction, `freqDeuxBitsConsecutifsEgaux(sequence)` qui calcule la fréquence de deux bits consécutifs égaux.
3. Tester vos fonctions : retrouvez-vous le résultat théorique ?

In [31]:
def generateurSignalArret(g1, g2, iCoeff1, iCoeff2, n) :
    l=[]
    p=0
    for _ in range(n):
        if(g1[0]==1):
            l.append(g2[0])
            g2=etatSuivant(g2,iCoeff2)[0]
        else:
            l.append(g2[0])
        g1 = etatSuivant(g1,iCoeff1)[0]
    return l

try:
    g1 = [1,0,1,0,1,1,0,0]
    g2 = [1,0,1,0,1,0,1,0]
    n = 20
    iCoeff1 = [0, 3, 5]
    iCoeff2 = [0, 2, 5, 6]
    s = generateurSignalArret(g1, g2, iCoeff1,iCoeff2, n)
    assert s == [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]
    print("generateurSignalArret : OK")
except:
    print("generateurSignalArret : ERREUR")


generateurSignalArret : OK


In [32]:
def freqDeuxBitsConsecutifsEgaux(sequence) : 
    p = 0
    for i in range(1,len(sequence)):
        if(sequence[i]==sequence[i-1]):
            p+=1
    return p/(len(sequence)-1)

try:
    s = [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]
    assert round(freqDeuxBitsConsecutifsEgaux(s),2) == 0.58 #ou 0.55
    print("freqDeuxBitsConsecutifsEgaux : OK")
except:
    print("freqDeuxBitsConsecutifsEgaux : ERREUR")


freqDeuxBitsConsecutifsEgaux : OK


In [33]:
iCoeff1 = [2,3,6]
iCoeff2 = [1,5]
p=0
for _ in range(500):
    g1 = list(npr.binomial(1, 1/2, 8)) #graine composée de huit 0 et 1 uniformément distribués 
    g2 = list(npr.binomial(1, 1/2, 8))
    p += freqDeuxBitsConsecutifsEgaux(generateurSignalArret(g1,g2,iCoeff1,iCoeff2,500))
print(p/500)


0.7498517034068142


*Remarque : si la fréquence n'est pas proche de 0.75, c'est que la période du générateur utilisé n'est pas assez grande pour approcher correctement la proba !*

> **Question 2 :**

> Vous avez intercepté le chiffré suivant :
<br>
<center>
$c = 1001\ 1000\ 1001\ 1100\ 0100\ 0001\ 0111\ 0000\ 0110\ 0100\ 1000\ 0001\ 1001\ 0110$
</center>
<br>
Vous savez qu'il s'agit du chiffrement de l'un des trois textes suivants à l'aide d'un __générateur à signal d'arrêt__ :
<br>
<center>
$m_1$ = "heyheyy",    
$m_2$ = "bonjour", 
$m_3$ = "hellllo" 
</center>
<br>
A l'aide des fonctions données ci-dessous, donner le texte clair qui a été le plus probablement chiffré.

In [34]:

def xor_lst(l1, l2) :
    """
    Cette fonction retourne le xor bit à bit entre deux listes binaires
    """
    res = []
    for x, y in zip(l1, l2) :
        res.append(x^y)
    return(res)

def chaineToListe(ch) :
    """
    Cette fonction transforme une chaine binaire (ex :"101") en une liste binaire (ex : [1,0,1])
    """
    return([int(x) for x in ch])

def listeToChaine(lst) :
    """
    Cette fonction transforme une liste binaire (ex : [1,0,1]) en une chaine binaire (ex :"101")
    """
    res = ""
    for i in range(len(lst)):
        res = res + str(lst[i])
    return res

def stringToBinary(msg):
    """
    Cette fonction transforme une chaine de caractere (ex : "ab") en une chaine binaire (ex :"0110000101100010")
    """
    msg_bin = ""
    for i in bytearray(msg, encoding ='ascii') :
        msg_bin = msg_bin + format(i, '08b')
    return msg_bin

def binaryToString(binary):
    """
    Cette fonction transforme une chaine binaire (ex : "0110000101100010") en une chaine de caractere (ex :"ab")
    """
    msg = ""
    for i in range(0, len(binary), 8):
        byte_int = int(binary[i:i+8], 2)
        byte_char = chr(byte_int)
        msg = msg + byte_char
        
    return msg

In [35]:
def nombre_plus_proche(valeur, liste):
    return min(liste, key=lambda x: abs(x - valeur))

c = chaineToListe("10011000100111000100000101110000011001001000000110010110")
m1 = chaineToListe(stringToBinary("heyheyy"))    
m2 = chaineToListe(stringToBinary("bonjour")) 
m3 = chaineToListe(stringToBinary("hellllo")) 

m1 = xor_lst(c, m1)
m2 = xor_lst(c, m2)
m3 = xor_lst(c, m3)

m1 = freqDeuxBitsConsecutifsEgaux(m1)
m2 = freqDeuxBitsConsecutifsEgaux(m2)
m3 = freqDeuxBitsConsecutifsEgaux(m3)

print("la solution est celle ayant la probabilité la plus proche de 0.75 : \n p(m1) = ", m1 ," p(m2) = ", m2," p(m3) = ", 3)


la solution est celle ayant la probabilité la plus proche de 0.75 : 
 p(m1) =  0.7454545454545455  p(m2) =  0.5454545454545454  p(m3) =  3


__Astuce__ : La suite chiffrante générée par le générateur à signal d'arret sert de clef pour chiffrer le message clair (par un xor bit à bit). Commencer par calculer les 3 suites chiffrantes candidates. L'une des trois a plus probablement été générée par un générateur à signal d'arret. Laquelle ? 

## Exercice 2 : le générateur de Geffe 

Le générateur de Geffe est un exemple de registres combinés (1973). Il est compoé de trois LFSR de longueurs distinctes combinés par la fonction :
<br>
<center>
$F(x_1,x_2,x_3) = x_1x_2 \oplus x_2x_3 \oplus x_3$
</center>
<br>

<img src="Geffe_TP3.png" width="700">

En 1985, T. Siegenthaler a proposé une méthode de cryptanalyse sur les systèmes de chiffrement par flot par registres combinés appelé *attaque par corrélation*. L'idée de cette méthode est d'exploiter l'existence d'une éventuelle corrélation entre la suite chiffrante et le contenu de certains registres. Un attaquant peut alors chercher séparément les valeurs initiales de chaque registre et retrouver la clé plus rapidement qu'en effectuant une recherche exhaustive. Le but de l'exercice est de montrer que le générateur de Geffe est vulnérable à une telle attaque.

> **Question 1 (à la main) :**

> En supposant que les valeurs $x_i(t)$ suivent des lois de Bernoulli indépendantes de paramètre $\frac{1}{2}$, montrer que :
<br>
<center>
$P\big(y(t) = x_1(t)\big) = \frac{3}{4} = P\big(y(t) = x_3(t)\big)$
</center>
<br>

> Pour cela, commencer par montrer que 
<br>
<center>
$y(t) = x_3(t)$ si $ x_2(t) = 0$
</center>
<br>
<br>
<center>
$y(t) = x_1(t)$ si $ x_2(t) = 1$ 
</center>
<br>

**Réponse :** 
Nous allons démontrer : $P\big(y(t) = x_1(t)\big) = \frac{3}{4} = P\big(y(t) = x_3(t)\big)$ grace à $F(x_1,x_2,x_3) = x_1x_2 \oplus x_2x_3 \oplus x_3$<br>
Pour se faire, nous allons dans un premier démontrer :
- $y(t) = x_3(t)$ si $ x_2(t) = 0$
- $y(t) = x_1(t)$ si $ x_2(t) = 1$ 

Pour $x_2 = 0$, nous avons : <br>
$y(t) = x_1 \times 0 \oplus 0 \times x_3 \oplus x_3$, nous obtenons donc: <br>
$y(t) = 0 \oplus 0 \oplus x_3 = x_3$

Pour $x_2 = 1$, nous avons : <br>
$y(t) = x_1 \times 1 \oplus 1 \times x_3 \oplus x_3$, nous obtenons donc: <br>
$y(t) = x_1 \oplus x_3 \oplus x_3$ or $x_3 \oplus x_3 =0$ donc $y(t) = x_1$

Nous avons 1 chance sur 2 que la sorti sois $x_1$ (de même pour $x_3$). Deplus, $P(x_1 == x_3) = \frac{1}{2}$ <br>
Donc $P(y(t) = x_1) = P(y(t) = x_1 |x_2=0) \times P(x_2=0) + P(y(t) = x_1 |x_2=1) \times P(x_2=1)$: <br>
$P(y(t) = x_1) = \frac{1}{2} \times \frac{1}{2} + 1 \times \frac{1}{2} = \frac{3}{4}$

> **Question 2 :**

1. Définir une fonction `generateurGeffe(g1, g2, g3, iCoeff1, iCoeff2, iCoeff3, n)` permettant de simuler un tel générateur. La fonction doit retourner les sorties des trois LFSR (afin de faciliter le calcul des fréquences) ainsi que la sortie du générateur complet.
2. Calculer les deux fréquences correspondant aux probabilités de la question précédente dans le cas du générateur dessiné ci-dessus avec des graines aléatoires. Retrouvez-vous les résultats théoriques ?

In [36]:
# 1

def generateurGeffe(g1, g2, g3, iCoeff1, iCoeff2, iCoeff3, n) :
    """
    Cette fonction retourne la liste des n bits générés à partir des graines g1, g2 et g3
    """
    x1 = suite_LFSR(g1,iCoeff1,n)
    x2 = suite_LFSR(g2,iCoeff2,n)
    x3 = suite_LFSR(g3,iCoeff3,n)
    y = []
    for i in range(n):
        if x2[i] == 1:
            y.append(x1[i])
        else:
            y.append(x3[i])
    return(x1, x2, x3, y)

try:
    iCoeff1 = [0,1]
    iCoeff2 = [1,2]
    iCoeff3 = [0,2]
    g1 = [1,1]
    g2 = [1,0,1]
    g3 = [1,0,0,1]
    n=5
    s = generateurGeffe(g1, g2, g3, iCoeff1, iCoeff2, iCoeff3, n)
    assert s == ([1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 1])
    print("generateurGeffe : OK")
except:
    print("generateurGeffe : ERREUR")


generateurGeffe : OK


In [37]:
def comparaison(l1,l2):
    p = 0
    for i in range(len(l1)):
        if l1[i] == l2[i]:
            p+=1
    return p / len(l1)

iCoeff1 = [0,2,13,17]
iCoeff2 = [8,16,20]
iCoeff3 = [5,7,13,16]
p1,p2=0,0
for _ in range(500):
    g1 = list(npr.binomial(1, 1/2, 19))
    g2 = list(npr.binomial(1, 1/2, 22))
    g3 = list(npr.binomial(1, 1/2, 17))
    x1,x2,x3,y = generateurGeffe(g1,g2,g3,iCoeff1,iCoeff2,iCoeff3,500)
    p1 += comparaison(x1,y)
    p2 += comparaison(x3,y)
print(p1/500)
print(p2/500)


0.7533440000000006
0.7455040000000003


> **Question 3 :**

En supposant connue la sortie du générateur, le nombre et la taille des LFSR ainsi que leur polynôme de rétroaction, proposer une attaque *par corrélation* pour déterminer la clé c'est à dire la graine ou encore les états initiaux de $R_1$, $R_2$ et $R_3$. __Les coefficients de rétroaction sont supposés connus.__

En cryptographie, le système de chiffrement est souvent public. La sécurité du système provient de la difficulté à retrouver la clef (ici la graine des registres).

_Attaque par corrélation :_ Cette attaque se base sur la connaissance du système de chiffrement et donc des deux probabilités calculées précédement. 
L’attaque consiste à déterminer par force brute (de manière exhaustive) le contenu initial du registre $R_1$ puis celui de $R_3$. 

Pour tous les contenus initiaux possibles de $R_1$, l’attaquant génère une suite de longueur $n$ pour ce registre. S'il s'agit du contenu initial recherché, le nombre de valeurs communes entre cette sortie de $R_1$ et celle du générateur de Geffe sera proche de $3n/4$. Une fois l’état initial de $R_1$ trouvé, l’attaquant peut répéter l’attaque pour obtenir celui de $R_3$.

Il ne reste plus qu'à déterminer le contenu initial de $R_2$ par force brute (en testant toutes les possibilités) en fonction des deux autres graines.

1. Définir une fonction `attaqueCorrelation(y, iCoeff, l)` qui prend en paramètre la suite chiffrante `y` générée par le générateur de Geffe, la liste `iCoeff` des coefficients de rétroaction du LFSR attaqué (i.e. $R_1$ ou $R_3$ ici) et la longueur `l` du registre attaqué et qui renvoie la graine la plus probable du LFSR. 
2. Définir une fonction `attaqueBruteForceR2(y, g1,g3, iCoeff1, iCoeff2, iCoeff3, l1, l2, l3)` qui cherche la graine de $R_2$ par force brute.
3. Tester votre attaque sur des exemples en vérifiant bien que les graines qui ont servies à générer la suite chiffrante sont les mêmes que celles que vous avez trouvé avec votre attaque.

In [38]:
def attaqueCorrelation(y, iCoeff, l): 
    lx2 = [[int(bit) for bit in bin(_)[2:].zfill(l)] for _ in range((l**2) -1)]
    res = []
    for x2 in lx2:
        res.append(comparaison(y,suite_LFSR(x2,iCoeff1,len(y))))
    return lx2[min(range(len(res)), key=lambda i: abs(res[i] - 0.75))]

def attaqueBruteForceR2(y, g1, g3, iCoeff1, iCoeff2, iCoeff3, l1, l2, l3):
    lx2 = [[int(bit) for bit in bin(_)[2:].zfill(l2)] for _ in range((l2**2) - 1)]
    solve = []
    for i in lx2:
        try:
            generated_y = generateurGeffe(g1, g3, iCoeff1, iCoeff2, iCoeff3, len(y))
            if generated_y == y:
                solve.append((g1, i, g3))  # Ajouter les paramètres qui correspondent à y
        except:
            pass
    return solve
    

In [39]:
iCoeff1 = [0, 2]
iCoeff2 = [1, 3, 4]
iCoeff3 = [5]
L = []
for i in range(100):
    y = list(np.random.binomial(1, 1/2, 6)) 
    l1, l2, l3 = 4, 5, 6 
    g1 = attaqueCorrelation(y, iCoeff1, l1)
    g3 = attaqueCorrelation(y, iCoeff3, l3)
    params = attaqueBruteForceR2(y, g1, g3, iCoeff1, iCoeff2, iCoeff3, l1, l2, l3)
    L.append(params)

print(L)

Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGeffe() missing 1 required positional argument: 'n'
Erreur: generateurGe

__Astuce pour `attaqueCorrelation`__ : 
On commence par creer la liste de toutes les graines possibles. Pour chaque graine à tester, on génére la suite chiffrante $x_1$ (ou $x_3$) de longueur $len(y)$. On compare cette suite à y. L'une des graines produira une suite plus pertinente que les autres...

Remarques :
>*Une attaque par force brute serait en $O\big(2^{l_1+l_2+l_3}\big)$* où $l_i$ désigne la longueur de $R_i$.<br>
>*L'attaque par corrélation demandée est en $O\big(2^{l_1}+2^{l_3}+2^{l_2}\big)$ voire en $O\big(2^{l_1}+2^{l_3}+l_2^3\big)$.*

>*D'autres attaques sont connues : *
>   - *l'attaque "deviner et déterminer" en $O\big(2^{l_2}\left(l_1^3+l_3^3\right)\big)$*
>   - *l'attaque algébrique en $O\big(\left(l_1+l_2+l_3\right)^6\big)$*